<a href="https://colab.research.google.com/github/Madeira-International-Workshop-in-ML/2023-prompt-engineering-chatbot/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot with OpenAI

In this notebook, we are going to implement, step-by-step, a chatbot using the OpenAI API.

Let's start by installing the required libraries.

In [ ]:
!pip install openai
!pip install gradio
!pip install python-dotenv

In [ ]:
from IPython.display import Markdown, display

def printmd(string):
    """
    Print markdown content in the notebook.
    :param string: the markdown content
    """
    display(Markdown(string))

## Load the environment variables

This step is similiar to what we implemented in the previous notebook. We will load the environment variables from the `.env` file.

In [ ]:
from dotenv import load_dotenv

In [ ]:
_ = load_dotenv('.env')

Now, let's load the OpenAI library, and set our API key.

In [ ]:
import openai
import os

In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY')

## Let's change the completion function

We will use a different completion function, but it is very similar to the one we used in the previous notebook.

In [ ]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0):
    """
    Get the completion from OpenAI API.
    :param messages: the messages to be sent to the chatbot
    :param model: the model to be used
    :param temperature: the temperature of the completion
    :return: the completion
    """
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    completion = response.choices[0].message["content"]
    return completion

## Roles

In OpenAI API, we can define the roles of the agents in the conversation.

In our case, we will have three roles: `system`, `user`, and `assistant`. The `system` role is used to set the behaviour of the chatbot (i.e., the  personality). The `user` role is the user that is interacting with the chatbot.  The `assistant` role is the chatbot itself, i.e., the chat model.

In ChatGPT web interface, your messages are the `user` messages, and then ChatGPT's are the `assistant` messages.

In [ ]:
messages = [
    {"role": "system", "content": "You are a teacher..."},
    {"role": "user",
     "content": "Explain how an artificial neural network works. Don't use more than 50 words."}
]

printmd(get_completion(messages))

_**Do it your self:**_ Try to change the `system` message to something else, and see how the chatbot behaves.

In [ ]:
messages = [
    {"role": "system", "content": "You are a children..."},
    {"role": "user",
     "content": "Explain how an artificial neural network works. Don't use more than 50 words."}
]

printmd(get_completion(messages))

_**Do it your self:**_ Now, stop here and think how can we continue the conversation. I.e., how can we make the chatbot continuing the conversation? How can the chatbot understand the context of the conversation?

In [ ]:
messages = [
    {"role": "system", "content": "You are a children..."},
    {"role": "user",
     "content": "Explain how an artificial neural network works. Don't use more than 50 words."},
    {"role": "assistant",
     "content": "An artificial neural network is like a brain made by computers. It has lots of tiny parts called neurons that work together to solve problems. It learns by practicing and adjusting its connections until it gets better at what it does."},
    {"role": "user",
     "content": "Can you give me more information about the functions of the neurons."}
]

printmd(get_completion(messages))

At this point, we can see what we need to do to make the chatbot continuing the conversation. We need to keep track of the conversation, and then use the last message to generate the next one. I.e., we need to keep track of the context of the conversation by storing the messages from the `user` and the `assistant`.

In [ ]:
context = [
    {"role": "system", "content": "You are an assitant..."}
]

In [ ]:
def chat(prompt, history=None):
    """
    Chat with the chatbot, and keep track of the context.
    :param prompt: the prompt to be sent to the chatbot
    :param history: dummy parameter for the gradio library
    :return: the completion
    """
    context.append({"role": "assistant", "content": prompt})
    completion = get_completion(context)
    context.append(
        {"role": "assistant", "content": completion})
    return completion

In [ ]:
printmd(chat(
    "Explain how an artificial neural network works. Don't use more than 50 words."))

In [ ]:
printmd(chat("Hum... it's still not very clear."))

In [ ]:
printmd(chat("Can you give me more information about the functions of the neurons."))

## Real-world example

In this example, we are going to read a text file with the information about the AI summit, and then use the chatbot to answer questions about the text.

In [ ]:
with open('info.md', 'r') as file:
    event_info = file.read()

In [ ]:
context = [
    {"role": "system",
     "content": "You are an organizer of the event, and you only reply to questions related to the event. Here is everything you need to know about the event: ```" + event_info + "```"}
]

In [ ]:
printmd(chat("What is the name of the event?"))

In [ ]:
printmd(chat("What is the date of the event?"))

In [ ]:
printmd(chat("What are the names of the speakers?"))

In [ ]:
printmd(chat("What is the name of the venue?"))

In [ ]:
printmd(chat("Is the event free?"))

In [ ]:
printmd(chat("Is it suitable for beginners in AI?"))

In [ ]:
printmd(chat("When is the coffee break?"))

## Chatbot web app

Using the gradio library, we can create a simple web app for our chatbot.

In [ ]:
import gradio as gr

In [ ]:
gr.ChatInterface(
    chat,
    chatbot=gr.Chatbot(),
    title="Welcome to the AI Summit chatbot",
    description="Ask me a question about the event...",
    theme="monochrome",
    examples=['how much does it cost?', 'what is the date of the event?',
              'what is the name of the venue?']
).launch(share=True)